# KNN Brute Force Approach
We first will explore KNN by looking at the brute force approach. In the brute force approach we do not use a special data structures to search through the data. We will simply calaculate the distance between every training vector and testing vector. As part of this exploration we will also look at algorithms for finding the K smallest numbers in an unordered collection of data.

To begin will use the distance metrics created in the Distance Metrics notebook and explain why they cause a bottle neck. Later we will create updated distance metrics in the Vectorized Distance Metrics notebook.

In [58]:
# Setup Some Test Data
import numpy as np
from knn.distance_metrics import euclidean

train_data = np.array([[1, 2, 3], [5, 6, 7], [8, 9, 10]])
test_data = np.array([[5, 10, 15], [10, 20, 30]])

print("Train Data: \n" + str(train_data))
print("Test Data: \n" + str(test_data))

Train Data: 
[[ 1  2  3]
 [ 5  6  7]
 [ 8  9 10]]
Test Data: 
[[ 5 10 15]
 [10 20 30]]


## KNN Brute Force - Full Sort To Get Smallest K
Here we use brute force KNN to find find all distances, then we fully sort the data to find the K smallest distances.
Sorting the data to find the k smallest items will result in a time complexity of O(nlog).

In [59]:
# Quick Prototype #

# Create A Distance Matrix. Each Row Represents A Test Vector And Each Column Is A Training Vector
# Note: We Will Store The Index of The Training Vector and Its Distance. This Makes Classification and Regression
#       Slightly Easier.
distance_matrix = np.zeros((test_data.shape[0], train_data.shape[0]), dtype=[("index",int), ("dist",float)])

for row, test_vector in enumerate(test_data):
    for col, train_vector in enumerate(train_data):
        distance_matrix[row, col] = (col, euclidean(test_vector, train_vector))
        
print(distance_matrix)

[[(0, 14.96662955) (1,  8.94427191) (2,  5.91607978)]
 [(0, 33.67491648) (1, 27.38612788) (2, 22.91287847)]]


In [60]:
# Sort To Find Smallest K
k = 2
sorted_distance_metric = np.sort(distance_matrix, order="dist")
smallest_k = sorted_distance_metric[:,:k]
print(smallest_k)

[[(2,  5.91607978) (1,  8.94427191)]
 [(2, 22.91287847) (1, 27.38612788)]]


In [61]:
# Make It All Into A Function
def knn_bf_full_sort(train_data, test_data , k=1):
    distance_matrix = np.zeros((test_data.shape[0], train_data.shape[0]), dtype=[("index",int), ("dist",float)])

    for row, test_vector in enumerate(test_data):
        for col, train_vector in enumerate(train_data):
            distance_matrix[row, col] = (col, euclidean(test_vector, train_vector))
            
    sorted_distance_metric = np.sort(distance_matrix, order="dist")
    smallest_k = sorted_distance_metric[:,:k]
    return smallest_k
    
print("KNN - Full Sort k=1: \n" + str(knn_bf_full_sort(train_data, test_data, 1)))
print("KNN - Full Sort k=2: \n" + str(knn_bf_full_sort(train_data, test_data, 2)))
print("KNN - Full Sort k=3: \n" + str(knn_bf_full_sort(train_data, test_data, 3)))

KNN - Full Sort k=1: 
[[(2,  5.91607978)]
 [(2, 22.91287847)]]
KNN - Full Sort k=2: 
[[(2,  5.91607978) (1,  8.94427191)]
 [(2, 22.91287847) (1, 27.38612788)]]
KNN - Full Sort k=3: 
[[(2,  5.91607978) (1,  8.94427191) (0, 14.96662955)]
 [(2, 22.91287847) (1, 27.38612788) (0, 33.67491648)]]


## KNN Brute Force - Partial Sort To Get Smallest K
Here we use brute force KNN to find find all distances, then we partial sort the data to get the smallest K. With partial sort we run K iterations of bubble sort to find top K, this results in a time complexity of O(kn).

In [62]:
# Quick Prototype #

# Create Some Data That Is The Same That WIll Be Encountered In Brute Force KNN
sample_sort_data = np.array([(0, 8), (1, 7), (2, 6), (3, 5), (4, 4), (5, 3), (6, 2), (7, 1)],
                            dtype=[("index",int), ("dist",float)])

k = 3
# Bubble Sort Is Ran Backwards To Push The Smallest Values To The Front
for i in range(0, k):
    for j in range(sample_sort_data.size-1, 0, -1):
        if sample_sort_data[j][1] < sample_sort_data[j-1][1]:
            sample_sort_data[[j, j-1]] = sample_sort_data[[j-1, j]]

smallest_k = sample_sort_data[:k]
print(smallest_k)


[(7, 1.) (6, 2.) (5, 3.)]


In [63]:
# Making Finding Smallest K Into Function
def smallest_k_partial_sort(array, k=1):
    for i in range(0, k):
        for j in range(array.size-1, 0, -1):
            if array[j][1] < array[j-1][1]:
                array[[j, j-1]] = array[[j-1, j]]

    top_k = array[:k]
    return top_k

print(smallest_k_partial_sort(sample_sort_data, k))


[(7, 1.) (6, 2.) (5, 3.)]


In [64]:
# Make It All Into A Function
def knn_bf_partial_sort(train_data, test_data ,k=1):
    distance_matrix = np.zeros((test_data.shape[0], train_data.shape[0]), dtype=[("index",int), ("dist",float)])

    for row, test_vector in enumerate(test_data):
        for col, train_vector in enumerate(train_data):
            distance_matrix[row, col] = (col, euclidean(test_vector, train_vector))
    
    smallest_k_matrix = np.zeros((test_data.shape[0], k), dtype=[("index",int), ("dist",float)])
    for i, row in enumerate(distance_matrix):
        smallest_k_matrix[i,:] = smallest_k_partial_sort(row, k)
        
    
    return smallest_k_matrix


print("KNN - Partial Sort k=1: \n" + str(knn_bf_partial_sort(train_data, test_data, 1)))
print("KNN - Partial Sort k=2: \n" + str(knn_bf_partial_sort(train_data, test_data, 2)))
print("KNN - Partial Sort k=3: \n" + str(knn_bf_partial_sort(train_data, test_data, 3)))

KNN - Partial Sort k=1: 
[[(2,  5.91607978)]
 [(2, 22.91287847)]]
KNN - Partial Sort k=2: 
[[(2,  5.91607978) (1,  8.94427191)]
 [(2, 22.91287847) (1, 27.38612788)]]
KNN - Partial Sort k=3: 
[[(2,  5.91607978) (1,  8.94427191) (0, 14.96662955)]
 [(2, 22.91287847) (1, 27.38612788) (0, 33.67491648)]]


## KNN Brute Force - Max-Heap To Get Top K

Here we use brute force KNN to find find all distances, then we will use a Max-Heap to find the smallest K. Finding smallest K will have a time complexity of O(nlogK).

In [65]:
# Quick Prototype #

import heapq 

k = 2

# Holds The Smallest K Data
smallest_k = np.array(np.zeros((test_data.shape[0], k)), dtype=[("index",int), ("dist",float)])

for row, test_vector in enumerate(test_data):
    
    max_heap = []
    
    for col, train_vector in enumerate(train_data):
        # Two Things To Note:
        # 1. heapq Only Offers a Min-Heap, So We Negate The Priorty Value To Make It A Max-Heap
        # 2. Tuples Are Compared By Their First Element, So We Switch The Typical Order Of Distance and Index
        dist = (-1.*euclidean(test_vector, train_vector), col)
        
        # Max-Heap Should Only Be Size K, We Need To Fill It Since Its Initially Empty
        if len(max_heap) < k:
            heapq.heappush(max_heap, dist)
        # If The Current Distance Is Less Than Max In Heap Then Remove Head And Add Current Distance
        # Remember We Have To Use Reverse Logic Due To Negateing Priority Value
        elif max_heap[0] < dist:
            heapq.heapreplace(max_heap, dist)
            
    # We Swap Order Of Index and Distance and Un-Negate Distance
    smallest_k[row,:] = [(x[1], -1.*x[0]) for x in max_heap]
    
    
print(smallest_k)


[[(1,  8.94427191) (2,  5.91607978)]
 [(1, 27.38612788) (2, 22.91287847)]]


In [66]:
def knn_bf_max_heap(train_data, test_data, k=1):
    smallest_k = np.array(np.zeros((test_data.shape[0], k)), dtype=[("index",int), ("dist",float)])

    for row, test_vector in enumerate(test_data):

        max_heap = []

        for col, train_vector in enumerate(train_data):
            # Two Things To Note:
            # 1. heapq Only Offers a Min-Heap, So We Negate The Priorty Value To Make It A Max-Heap
            # 2. Tuples Are Compared By Their First Element, So We Switch The Typical Order Of Distance and Index
            dist = (-1.*euclidean(test_vector, train_vector), col)

            # Max-Heap Should Only Be Size K, We Need To Fill It Since Its Initially Empty
            if len(max_heap) < k:
                heapq.heappush(max_heap, dist)
            # If The Current Distance Is Less Than Max In Heap Then Remove Head And Add Current Distance
            # Remember We Have To Use Reverse Logic Due To Negateing Priorty Value
            elif max_heap[0] < dist:
                heapq.heapreplace(max_heap, dist)

        # We Swap Order Of Index and Distance and Un-Negate Distance
        smallest_k[row,:] = [(x[1], -1.*x[0]) for x in max_heap]
        
    return smallest_k

print("KNN - Max Heap k=1: \n" + str(knn_bf_max_heap(train_data, test_data, 1)))
print("KNN - Max Heap k=2: \n" + str(knn_bf_max_heap(train_data, test_data, 2)))
print("KNN - Max Heap k=3: \n" + str(knn_bf_max_heap(train_data, test_data, 3)))

KNN - Max Heap k=1: 
[[(2,  5.91607978)]
 [(2, 22.91287847)]]
KNN - Max Heap k=2: 
[[(1,  8.94427191) (2,  5.91607978)]
 [(1, 27.38612788) (2, 22.91287847)]]
KNN - Max Heap k=3: 
[[(0, 14.96662955) (1,  8.94427191) (2,  5.91607978)]
 [(0, 33.67491648) (1, 27.38612788) (2, 22.91287847)]]


## KNN Brute Force - Median of Medians To Find Top K
Here we use brute force KNN to find find all distances, then we will use the Median of Medians algorithm to find the smallest K. Finding smallest K will have a time complexity of O(n).

### Median of Medians
Median of Medians is an selection algorithm that finds the kth order statistic of an unordered list in O(n) time complexity. To find the smallest k, we will find the kth order statistic and then find all elements equal to or smaller than that order statistic.

In [67]:
def medians_of_medians(array, k=0):
    
    # Base Case When Small Array Is Given - Just Find Median
    if array.size <= 5:
        return np.sort(array)[int(k)]
    
    # Make Lists Of Size 5 From Original Input    
    subsets_size_five = np.array(list(map(lambda x: array[x:x+5], range(0, array.size, 5))))
    
    # Find The Median Of Every Sublist
    # If Sublist Even Return One Of The Two Numbers
    median_of_subsets = np.array(list(map(lambda x: np.sort(x)[int(x.size/2)], subsets_size_five)))
    
    # Find Median Of List Of Medians
    mom = medians_of_medians(median_of_subsets, int(median_of_subsets.size/2))
    
    # Make Two Lists: Numbers Greater Than Median Of Medians and Less Than Median Of Medians
    left = array[array < mom]
    right = array[array > mom]
    
    # Position of Median Of Medians
    pos_of_mom = left.size
    
    # Explore Numbers To Left or Right Unless We Found The Kth Order Statistic
    if pos_of_mom < k:
        return medians_of_medians(right, k-pos_of_mom-1 )
    elif pos_of_mom > k:
        return medians_of_medians(left, k)
        
    return mom
    
print("Note Order Statistics Start At 0 And End at n-1")
print("Data: " + str(np.array(range(17, 0, -1))))
print("0 Order Statistic: " + str(medians_of_medians(np.array(range(17, 0, -1)), 0)))
print("3rd Order Statistic: " + str(medians_of_medians(np.array(range(17, 0, -1)), 3)))
print("8th Order Statistic: " + str(medians_of_medians(np.array(range(17, 0, -1)), 8)))
print("14th Order Statistic: " + str(medians_of_medians(np.array(range(17, 0, -1)), 14)))
print("16th Order Statistic: " + str(medians_of_medians(np.array(range(17, 0, -1)), 16)))


Note Order Statistics Start At 0 And End at n-1
Data: [17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1]
0 Order Statistic: 1
3rd Order Statistic: 4
8th Order Statistic: 9
14th Order Statistic: 15
16th Order Statistic: 17


In [68]:
from timeit import default_timer as timer

# The Medians of Medians Algoirhtm Must Be Update To Work With The Paired Data We Are Using
# Remember In The Distance Matrix We Store The Training Vector Index and The Distance

        

def medians_of_medians(array, k=0):
    
    # Base Case When Small Array Is Given - Just Find Median
    if array.size <= 5:
        return np.sort(array, order="dist")[int(k)]
    
    # Make Lists Of Size 5 From Original Input    
    subsets_size_five = np.array(list(map(lambda x: array[x:x+5], range(0, int(array.size), 5))))
    
    # Find The Median Of Every Sublist
    # If Sublist Even Return One Of The Two Numbers
    median_of_subsets = np.array(list(map(lambda x: np.sort(x, order="dist")[int(x.size/2)], subsets_size_five)))
    
    # Find Median Of List Of Medians
    mom = medians_of_medians(median_of_subsets, int(median_of_subsets.size/2))
    
    # Make Two Lists: Numbers Greater Than Median Of Medians and Less Than Median Of Medians    
    left = array[array["dist"] < mom["dist"]]
    right = array[array["dist"] > mom["dist"]]
    
    # Position of Median Of Medians
    pos_of_mom = left.size

    # Explore Numbers To Left or Right Unless We Found The Kth Order Statistic
    if pos_of_mom < k:
        return medians_of_medians(right, k-pos_of_mom-1 )
    elif pos_of_mom > k:
        return medians_of_medians(left, k)
        
    return mom

sample_data = np.array([(0, 8), (1, 7), (2, 6), (3, 5), (4, 4), (5, 3), (6, 2), (7, 1)],
                       dtype=[("index",int), ("dist",float)])

print("Data:\n"+ str(sample_data))
print("(index, distance)")
print("0 Order Statistic: " + str(medians_of_medians(sample_data, 0)))
print("3rd Order Statistic: " + str(medians_of_medians(sample_data, 3)))
print("4th Order Statistic: " + str(medians_of_medians(sample_data, 4)))
print("6th Order Statistic: " + str(medians_of_medians(sample_data, 6)))
print("7th Order Statistic: " + str(medians_of_medians(sample_data, 7)))

Data:
[(0, 8.) (1, 7.) (2, 6.) (3, 5.) (4, 4.) (5, 3.) (6, 2.) (7, 1.)]
(index, distance)
0 Order Statistic: (7, 1.)
3rd Order Statistic: (4, 4.)
4th Order Statistic: (3, 5.)
6th Order Statistic: (1, 7.)
7th Order Statistic: (0, 8.)


In [69]:
# Making Finding Smallest K Into Function
def smallest_k_median_of_medians(array, k=1):
    kth_order_stat = medians_of_medians(array, k-1)
    return array[array["dist"] <= kth_order_stat["dist"]]

print(smallest_k_median_of_medians(sample_data, 2))


[(6, 2.) (7, 1.)]


In [70]:
# Make It All Into A Function
def knn_bf_median_of_medians(train_data, test_data ,k=1):
    distance_matrix = np.zeros((test_data.shape[0], train_data.shape[0]), dtype=[("index",int), ("dist",float)])

    for row, test_vector in enumerate(test_data):
        for col, train_vector in enumerate(train_data):
            distance_matrix[row, col] = (col, euclidean(test_vector, train_vector))
    
    smallest_k_matrix = np.zeros((test_data.shape[0], k), dtype=[("index",int), ("dist",float)])

    for i, row in enumerate(distance_matrix):
        smallest_k_matrix[i,:] = smallest_k_median_of_medians(row, k)
        
    return smallest_k_matrix

print("KNN - Median of Medians k=1: \n" + str(knn_bf_median_of_medians(train_data, test_data, 1)))
print("KNN - Median of Medians k=2: \n" + str(knn_bf_median_of_medians(train_data, test_data, 2)))
print("KNN - Median of Medians k=3: \n" + str(knn_bf_median_of_medians(train_data, test_data, 3)))

KNN - Median of Medians k=1: 
[[(2,  5.91607978)]
 [(2, 22.91287847)]]
KNN - Median of Medians k=2: 
[[(1,  8.94427191) (2,  5.91607978)]
 [(1, 27.38612788) (2, 22.91287847)]]
KNN - Median of Medians k=3: 
[[(0, 14.96662955) (1,  8.94427191) (2,  5.91607978)]
 [(0, 33.67491648) (1, 27.38612788) (2, 22.91287847)]]


## Intro Select

Uses Quick-Select to begin with, but switches to Median of Medians if recursion goes too deep. Gives faster average speed time while in worst case ensuring O(n) time.

In [71]:
def knn_bf_introselect(train_data, test_data ,k=1):
    distance_matrix = np.zeros((test_data.shape[0], train_data.shape[0]), dtype=[("index",int), ("dist",float)])

    for row, test_vector in enumerate(test_data):
        for col, train_vector in enumerate(train_data):
            distance_matrix[row, col] = (col, euclidean(test_vector, train_vector))
    
    smallest_k_matrix = np.zeros((test_data.shape[0], k), dtype=[("index",int), ("dist",float)])
    
    for i, row in enumerate(distance_matrix):
        smallest_k_matrix[i,:] = np.partition(row, k-1, order="dist")[:k]
        
    return smallest_k_matrix

print("KNN - Introselect k=1: \n" + str(knn_bf_introselect(train_data, test_data, 1)))
print("KNN - Introselect k=2: \n" + str(knn_bf_introselect(train_data, test_data, 2)))
print("KNN - Introselect k=3: \n" + str(knn_bf_introselect(train_data, test_data, 3)))

KNN - Introselect k=1: 
[[(2,  5.91607978)]
 [(2, 22.91287847)]]
KNN - Introselect k=2: 
[[(2,  5.91607978) (1,  8.94427191)]
 [(2, 22.91287847) (1, 27.38612788)]]
KNN - Introselect k=3: 
[[(2,  5.91607978) (1,  8.94427191) (0, 14.96662955)]
 [(2, 22.91287847) (1, 27.38612788) (0, 33.67491648)]]


### Quick Aside - Finding Median of A List of 5 Numbers Using 6 Comparisions
When reading about median of medians here(https://www.ics.uci.edu/~eppstein/161/960130.html) the author mentions that it is possible to find the median of five elements with only six comparisons. This is more optimal compared to 
performing a full sort on all subsets. Out of pure interest I imeplement this here but do not use it in my code.

In [72]:
def median_five_elements(array):
    # Compare Index 0 and 1 And Possibly Swap
    # One
    if array[0]["dist"] > array[1]["dist"]:
        array[[0, 1]] = array[[1, 0]]
        
    # Compare Index 2 and 3 And Possibly Swap
    # Twon
    if array[2]["dist"] > array[3]["dist"]:
        array[[2, 3]] = array[[3, 2]]
        
    # Three
    if array[2]["dist"] < array[0]["dist"]:
        array[[1, 3]] = array[[3, 1]]
        array[[0, 2]] = array[[2, 0]]
        
    array[[0, 4]] = array[[4, 0]]
    
    # Four
    if array[0]["dist"] > array[1]["dist"]:
        array[[0, 1]] = array[[1, 0]]
    
    # Five
    if array[2]["dist"] > array[0]["dist"]:
        array[[1, 3]] = array[[3, 1]]
        array[[0, 2]] = array[[2, 0]]
        
    # Six
    if array[0]["dist"] > array[3]["dist"]:
        return array[3]
    
    return array[0]

## Benchmarking


In [73]:
# Random Data
rand_train = np.random.randn(5000,100)
rand_test = np.random.randn(500,100)

In [74]:
%time run_1 = knn_bf_full_sort(rand_train, rand_test, 1)
%time run_2 = knn_bf_full_sort(rand_train, rand_test, 3)
%time run_3 = knn_bf_full_sort(rand_train, rand_test, 5)
%time run_4 = knn_bf_full_sort(rand_train, rand_test, 7)

CPU times: user 9.9 s, sys: 27.9 ms, total: 9.93 s
Wall time: 9.93 s
CPU times: user 10.6 s, sys: 47.4 ms, total: 10.6 s
Wall time: 10.6 s
CPU times: user 9.98 s, sys: 34.2 ms, total: 10 s
Wall time: 10 s
CPU times: user 9.83 s, sys: 38.4 ms, total: 9.87 s
Wall time: 9.86 s


In [75]:
%time run_1 = knn_bf_partial_sort(rand_train, rand_test, 1)
%time run_2 = knn_bf_partial_sort(rand_train, rand_test, 3)
%time run_3 = knn_bf_partial_sort(rand_train, rand_test, 5)
%time run_4 = knn_bf_partial_sort(rand_train, rand_test, 7)

CPU times: user 20.4 s, sys: 32.3 ms, total: 20.5 s
Wall time: 20.5 s
CPU times: user 43.3 s, sys: 45.8 ms, total: 43.4 s
Wall time: 43.4 s
CPU times: user 1min 6s, sys: 54.2 ms, total: 1min 6s
Wall time: 1min 6s
CPU times: user 1min 33s, sys: 159 ms, total: 1min 33s
Wall time: 1min 33s


In [76]:
%time run_1 = knn_bf_max_heap(rand_train, rand_test, 1)
%time run_2 = knn_bf_max_heap(rand_train, rand_test, 3)
%time run_3 = knn_bf_max_heap(rand_train, rand_test, 5)
%time run_4 = knn_bf_max_heap(rand_train, rand_test, 7)

CPU times: user 8.68 s, sys: 16.8 ms, total: 8.7 s
Wall time: 8.69 s
CPU times: user 8.68 s, sys: 20.3 ms, total: 8.7 s
Wall time: 8.69 s
CPU times: user 8.73 s, sys: 19.5 ms, total: 8.75 s
Wall time: 8.74 s
CPU times: user 8.66 s, sys: 13.9 ms, total: 8.67 s
Wall time: 8.67 s


In [77]:
%time run_1 = knn_bf_median_of_medians(rand_train, rand_test, 1)
%time run_2 = knn_bf_median_of_medians(rand_train, rand_test, 3)
%time run_3 = knn_bf_median_of_medians(rand_train, rand_test, 5)
%time run_4 = knn_bf_median_of_medians(rand_train, rand_test, 7)

CPU times: user 25.4 s, sys: 74.3 ms, total: 25.4 s
Wall time: 25.4 s
CPU times: user 24.8 s, sys: 67.3 ms, total: 24.8 s
Wall time: 24.8 s
CPU times: user 24.7 s, sys: 41.7 ms, total: 24.7 s
Wall time: 24.7 s
CPU times: user 24.5 s, sys: 36.8 ms, total: 24.6 s
Wall time: 24.6 s


In [78]:
%time run_1 = knn_bf_introselect(rand_train, rand_test, 1)
%time run_2 = knn_bf_introselect(rand_train, rand_test, 3)
%time run_3 = knn_bf_introselect(rand_train, rand_test, 5)
%time run_4 = knn_bf_introselect(rand_train, rand_test, 7)

CPU times: user 9.86 s, sys: 19.4 ms, total: 9.88 s
Wall time: 9.87 s
CPU times: user 9.77 s, sys: 15.7 ms, total: 9.78 s
Wall time: 9.78 s
CPU times: user 9.81 s, sys: 27.3 ms, total: 9.83 s
Wall time: 9.82 s
CPU times: user 9.9 s, sys: 22.9 ms, total: 9.93 s
Wall time: 9.92 s
